In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
batch_size = 64
test_batch_size = 1000
epochs = 5
lr = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])),
                   batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False,transform=transforms.Compose([transforms.ToTensor()])),
                   batch_size=test_batch_size, shuffle=True)

In [6]:
for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.338169
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.150904
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.120628
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.140503
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.137385
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.091047
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.285761
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.111738
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.214592
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.356457
Test set: Average loss: 0.1462, Accuracy: 9560/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.220217
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.171510
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.303528
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.299989
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.282878
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.360289
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.164971
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.160066
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.092216
Tr